# Example of running PhaseLink

In [2]:
# Specify if running in google colab:
use_google_colab = True

# Install/add neccessary paths if using colab:
if use_google_colab:
    !pip install obspy
    # Install nvidia-apex:
    !git clone https://github.com/NVIDIA/apex
    !pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
    

In [7]:
# Import neccessary modules:
import sys, shutil
import json
import multiprocessing as mp
import pickle
import numpy as np 
import torch
import gc 
from obspy.geodetics.base import gps2dist_azimuth
if not use_google_colab:
  %load_ext autoreload
  %autoreload 2
# And import PhaseLink:
if use_google_colab:
  shutil.rmtree('./PhaseLink', ignore_errors=True)
  !git clone https://github.com/TomSHudson/PhaseLink.git
  sys.path.append('./PhaseLink/')
else:
  sys.path.append('..')
import phaselink_dataset
import phaselink_train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Copy over example files into pwd if using colab:
if use_google_colab:
  !cp PhaseLink/example/params.json .
  !cp PhaseLink/example/station_list.txt .
  !cp PhaseLink/example/tt.pg .
  !cp PhaseLink/example/tt.sg .

## 0. Load in param file and key info:

In [6]:
# Import param json file:
params_fname = "params.json"
with open(params_fname, "r") as f:
    params = json.load(f)
    
# Get GPU info if using colab:
if use_google_colab:
     print("GPU:", torch.cuda.get_device_name(0))
     params['device'] = "cuda:0" # Use first GPU

## 1. Create a synthetic training dataset:

In [4]:
# Set key parameters from param file:
max_picks = params['n_max_picks']
t_max = params['t_win']
n_threads = params['n_threads']

print("Starting up...")
# Setup grid:
phase_idx = {'P': 0, 'S': 1}
lat0, lon0 = phaselink_dataset.get_network_centroid(params)
stlo, stla, phasemap, sncl_idx, stations, sncl_map = \
    phaselink_dataset.build_station_map(params, lat0, lon0, phase_idx)
x_min = np.min(stlo)
x_max = np.max(stlo)
y_min = np.min(stla)
y_max = np.max(stla)
for key in sncl_map:
    X0, Y0 = stations[key]
    X0 = (X0 - x_min) / (x_max - x_min)
    Y0 = (Y0 - y_min) / (y_max - y_min)
    stations[key] = (X0, Y0)

# Save station maps for detect mode
pickle.dump(stations, open(params['station_map_file'], 'wb'))
pickle.dump(sncl_map, open(params['sncl_map_file'], 'wb'))

# # Pwaves
# pTT = tt_interp(params['tt_table']['P'], params['datum'])
# print('Read pTT')
# print('(dep,dist) = (0,0), (10,0), (0,10), (10,0):')
# print('             {:.3f}   {:.3f}   {:.3f}   {:.3f}'.format(
# pTT.interp(0,0), pTT.interp(10,0),pTT.interp(0,10),
# pTT.interp(10,10)))

# #Swaves
# sTT = tt_interp(params['tt_table']['S'], params['datum'])
# print('Read sTT')
# print('(dep,dist) = (0,0), (10,0), (0,10), (10,0):')
# print('             {:.3f}   {:.3f}   {:.3f}   {:.3f}'.format(
# sTT.interp(0,0), sTT.interp(10,0),sTT.interp(0,10),
# sTT.interp(10,10)))

# Get travel-time tables for P and S waves:
pTT = phaselink_dataset.tt_interp(params['trav_time_p'], params['datum'])
sTT = phaselink_dataset.tt_interp(params['trav_time_s'], params['datum'])

# Generate synthetic training dataset for given param file:
in_q = mp.Queue() ###1000000)
out_q = mp.Queue() ###1000000)
proc = mp.Process(target=phaselink_dataset.output_thread, args=(out_q, params))
proc.start()
procs = []
for i in range(n_threads):
    print("Starting thread %d" % i)
    p = mp.Process(target=phaselink_dataset.generate_phases, \
        args=(in_q, out_q, x_min, x_max, y_min, y_max, \
              sncl_idx, stla, stlo, phasemap, pTT, sTT, params))
    p.start()
    procs.append(p)

for i in range(params['n_train_samp']):
    in_q.put(i)

for i in range(n_threads):
    in_q.put(None)
#for p in procs:
#    p.join()
#proc.join()

print("Creating the following files for the PhaseLink synthetic training dataset:")
print(params['station_map_file'])
print(params['sncl_map_file'])
print(params['training_dset_X'])
print(params['training_dset_Y'])

0
Starting up...
Starting thread 0
Starting thread 1
Starting thread 2
Starting thread 3
Starting thread 4
Starting thread 5
Starting thread 6
Starting thread 7
Creating the following files for the PhaseLink synthetic training dataset:
station_map.pkl
sncl_map.pkl
shimane_train_X.npy
shimane_train_Y.npy
P-phases (zeros): 11862944 ( 98.0246570815 %)
S-phases (ones): 239056 ( 1.97534291853 %)
Saved the synthetic training dataset.


## 2. Train the model using the syntehetic dataset:

In [38]:
# Get device (cpu vs gpu) specified:
device = torch.device(params["device"])
if params["device"][0:4] == "cuda":
    torch.cuda.empty_cache()
    enable_amp = True
else:
    enable_amp = False
if enable_amp:
    import apex.amp as amp

# Get training info from param file:
n_epochs = params["n_epochs"] #100

# Load in training dataset:
X = np.load(params["training_dset_X"])
Y = np.load(params["training_dset_Y"])
print("Training dataset info:")
print("Shape of X:", X.shape, "Shape of Y", Y.shape)
dataset = phaselink_train.MyDataset(X, Y, device)

# Get dataset info:
n_samples = len(dataset)
indices = list(range(n_samples))

# Set size of training and validation subset:
n_test = int(0.1*X.shape[0])
validation_idx = np.random.choice(indices, size=n_test, replace=False)
train_idx = list(set(indices) - set(validation_idx))

# Specify samplers:
train_sampler = phaselink_train.SubsetRandomSampler(train_idx)
validation_sampler = phaselink_train.SubsetRandomSampler(validation_idx)

# Load training data:
train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=False,
    sampler=train_sampler
)
val_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1024,
    shuffle=False,
    sampler=validation_sampler
)

stackedgru = phaselink_train.StackedGRU()
stackedgru = stackedgru.to(device)
#stackedgru = torch.nn.DataParallel(stackedgru,
#    device_ids=['cuda:2', 'cuda:3', 'cuda:4', 'cuda:5'])

if enable_amp:
    #amp.register_float_function(torch, 'sigmoid')
    from apex.optimizers import FusedAdam
    optimizer = FusedAdam(stackedgru.parameters())
    stackedgru, optimizer = amp.initialize(
        stackedgru, optimizer, opt_level='O2')
else:
    optimizer = torch.optim.Adam(stackedgru.parameters())

model = phaselink_train.Model(stackedgru, optimizer, \
    model_path='./phaselink_model/')
print("Begin training process.")
model.train(train_loader, val_loader, n_epochs, enable_amp=enable_amp)


Training dataset info:
Shape of X: (8054, 1500, 5) Shape of Y (8054, 1500)
Begin training process.
Epoch 1, 10% 	 train_loss: 9.1909e-01 train_acc: 66.02% took: 39.16s
Epoch 1, 20% 	 train_loss: 5.1903e-01 train_acc: 81.98% took: 36.38s
Epoch 1, 31% 	 train_loss: 2.2365e-01 train_acc: 87.30% took: 36.89s
Epoch 1, 41% 	 train_loss: 1.3750e-01 train_acc: 90.02% took: 36.47s
Epoch 1, 51% 	 train_loss: 1.7938e-01 train_acc: 91.62% took: 36.45s
Epoch 1, 62% 	 train_loss: 1.9227e-01 train_acc: 92.69% took: 36.98s
Epoch 1, 72% 	 train_loss: 1.9307e-01 train_acc: 93.43% took: 36.83s
Epoch 1, 82% 	 train_loss: 1.8742e-01 train_acc: 93.98% took: 36.95s
Epoch 1, 93% 	 train_loss: 1.3702e-01 train_acc: 94.45% took: 36.22s
Precision (Class 0): 0.980%
Recall (Class 0): 1.000%


ZeroDivisionError: division by zero